In [18]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
SRC_PATH = PROJECT_ROOT / "src"
sys.path.append(str(SRC_PATH))

import numpy as np
import pandas as pd


In [19]:
# df_roll should already exist if run after 04
df = df_roll.copy()
df = df.set_index("date").sort_index()

df.head()


,cycle_strength,window,m,tau,representation,cyclic_regime
date,,,,,,
1819-12-31,0.000000,107,12,3,log_price,Non-Cyclic
1820-03-31,0.000000,107,12,3,log_price,Non-Cyclic
1820-03-31,0.012543,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.012543,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.000000,107,12,3,log_price,Non-Cyclic


In [20]:
def load_macro(filename, value_col):
    path = PROJECT_ROOT / "data/raw" / filename
    df = pd.read_csv(path, parse_dates=["Date"])
    
    # Normalize to end-of-month to match XAU_Monthly
    df["Date"] = df["Date"] + pd.offsets.MonthEnd(0)
    
    return df.set_index("Date")[value_col]


In [21]:
cpi = load_macro("CPI_Monthly.csv", "CPI")
m2  = load_macro("M2_Monthly.csv", "M2")
ff  = load_macro("FEDFUNDS_Monthly.csv", "FedFunds")


In [22]:
df["CPI"] = cpi
df["M2"] = m2
df["FedFunds"] = ff

df.tail()


,cycle_strength,window,m,tau,representation,cyclic_regime,CPI,M2,FedFunds
date,,,,,,,,,
2025-10-31,0.046153,107,12,3,log_price,Non-Cyclic,NaN,22298.0,4.09
2025-11-30,0.010402,107,12,3,log_returns,Non-Cyclic,325.031,22322.4,3.88
2025-11-30,0.046153,107,12,3,log_price,Non-Cyclic,325.031,22322.4,3.88
2025-12-31,0.046153,107,12,3,log_price,Non-Cyclic,NaN,NaN,NaN
2025-12-31,0.010402,107,12,3,log_returns,Non-Cyclic,NaN,NaN,NaN


In [23]:
last_macro_date = df[["CPI", "M2", "FedFunds"]].dropna().index.max()

df = df.loc[:last_macro_date].copy()

last_macro_date, df.tail()


(Timestamp('2025-11-30 00:00:00'),
             cycle_strength  window   m  tau representation cyclic_regime  \
 date                                                                       
 2025-09-30        0.046153     107  12    3      log_price    Non-Cyclic   
 2025-10-31        0.009246     107  12    3    log_returns    Non-Cyclic   
 2025-10-31        0.046153     107  12    3      log_price    Non-Cyclic   
 2025-11-30        0.010402     107  12    3    log_returns    Non-Cyclic   
 2025-11-30        0.046153     107  12    3      log_price    Non-Cyclic   
 
                 CPI       M2  FedFunds  
 date                                    
 2025-09-30  324.368  22212.4      4.22  
 2025-10-31      NaN  22298.0      4.09  
 2025-10-31      NaN  22298.0      4.09  
 2025-11-30  325.031  22322.4      3.88  
 2025-11-30  325.031  22322.4      3.88  )

In [24]:
df["inflation_yoy"] = df["CPI"].pct_change(12)
df["inflation_accel"] = df["inflation_yoy"].diff()

df["inflation_regime"] = np.where(
    df["inflation_accel"] > 0,
    "Accelerating",
    "Decelerating"
)


/var/folders/qt/8016q_0d4kb4ly1txywph44r0000gn/T/ipykernel_23563/1899793426.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["inflation_yoy"] = df["CPI"].pct_change(12)


In [25]:
df["m2_yoy"] = df["M2"].pct_change(12)

df["liquidity_regime"] = np.where(
    df["m2_yoy"] > df["m2_yoy"].median(),
    "Loose",
    "Tight"
)


In [26]:
df["rate_regime"] = np.where(
    df["FedFunds"] > df["FedFunds"].median(),
    "High",
    "Low"
)


In [27]:
threshold = df["cycle_strength"].quantile(0.95)

df["cyclic_regime"] = np.where(
    df["cycle_strength"] >= threshold,
    "Cyclic",
    "Non-Cyclic"
)

df["cyclic_regime"].value_counts()


cyclic_regime
Non-Cyclic    2441
Cyclic         134
Name: count, dtype: int64

In [28]:
pd.crosstab(
    df["inflation_regime"],
    df["cyclic_regime"],
    normalize="index"
)


cyclic_regime,Cyclic,Non-Cyclic
inflation_regime,,
Accelerating,0.055233,0.944767
Decelerating,0.051546,0.948454


In [29]:
pd.crosstab(
    df["liquidity_regime"],
    df["cyclic_regime"],
    normalize="index"
)


cyclic_regime,Cyclic,Non-Cyclic
liquidity_regime,,
Loose,0.069061,0.930939
Tight,0.045381,0.954619


In [30]:
pd.crosstab(
    df["rate_regime"],
    df["cyclic_regime"],
    normalize="index"
)


cyclic_regime,Cyclic,Non-Cyclic
rate_regime,,
High,0.057641,0.942359
Low,0.049754,0.950246


In [31]:
df.groupby("inflation_regime")["cycle_strength"].mean()
df.groupby("liquidity_regime")["cycle_strength"].mean()
df.groupby("rate_regime")["cycle_strength"].mean()


rate_regime
High    0.026402
Low     0.018527
Name: cycle_strength, dtype: float64

In [32]:
out_path = PROJECT_ROOT / "powerbi/exports/PH_XAU_8p88y_MacroConditioned.csv"

df.reset_index().to_csv(out_path, index=False)

out_path


PosixPath('/Users/Rod/Desktop/cycle-tda-lab/powerbi/exports/PH_XAU_8p88y_MacroConditioned.csv')